In [1]:
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"    # 日志格式化输出
DATE_FORMAT = "%m/%d/%Y %H:%M:%S %p"                        # 日期格式
fp = logging.FileHandler('esio.log')
fs = logging.StreamHandler()
logging.basicConfig(level=logging.WARNING, format=LOG_FORMAT, datefmt=DATE_FORMAT, handlers=[fs])    # 调用

import pandas as pd
from pandasticsearch import DataFrame, Select
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk, parallel_bulk
from tqdm import tqdm

from sklearn import datasets

from tool.sink import ESSink

In [2]:
df = datasets.load_iris()
df = pd.DataFrame(df.data, columns=['x1','x2','x3','x4'])
df.head(4)

,x1,x2,x3,x4
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2


In [3]:
es_localhost = "192.168.3.248:9200"
es_ysyp_test = "115.29.34.243:9200"
unsername = 'elastic'
password = 'superzsh123'

es = Elasticsearch(
    [es_ysyp_test],
    http_auth=(unsername, password)
)

In [4]:
%%time
user_feature = ESSink(df, es)
user_feature.to_es("user_feature")

CPU times: user 27.6 ms, sys: 11.4 ms, total: 39 ms
Wall time: 397 ms


In [5]:
%%time
# result_df = DataFrame.from_es(url='http://115.29.34.243:9200', username=unsername, password=password, index='index_bbs_browse_record', doc_type='type_bbs_browse_record', compat=5).to_pandas()
click_dict = es.search(index="index_bbs_browse_record", body={"query": {"match_all": {}}, "size":1000}, timeout='30m')
click_df = Select.from_dict(click_dict).to_pandas().drop(['_index', '_type', '_id', '_score'], axis=1)

pv_dict = es.search(index="index_syp_user_suggest", body={"query": {"match_all": {}}, "size":1000}, timeout='30m')
pv_df = Select.from_dict(pv_dict).to_pandas().drop(['_index', '_type', '_id', '_score'], axis=1)

CPU times: user 37.8 ms, sys: 10.4 ms, total: 48.1 ms
Wall time: 237 ms


In [6]:
click_df

,createtime,bbsprice,partitionId,maxValue,partitiontitle,brand_name,title,userid,parentid,brand_id,uid,times,minValue,parentidtitle,businessauthentication,bbsUserid,updatetime,bbsid,status,browseSource
0,2019-07-31 17:22:41,1.0,2017111154,3001.0,女表,,cartier 卡地亚手表,1038111778,2017111152,0,11447,1,0.0,,0,1038111617,2019-07-31 17:22:41,20172944,0.0,NaN
1,2019-08-05 11:08:14,66666.0,2017111190,1000000.0,斜挎包,,乔丹 大喊大叫大家觉得计算机等级,1038111556,2017111137,0,11869,1,46666.0,,0,1098119585,2019-08-05 11:08:14,20174327,0.0,NaN
2,2019-08-07 16:03:09,665.0,2017111190,3665.0,斜挎包,,爱彼 你就提吉里吉里经理,1038111556,2017111137,0,12034,1,0.0,,0,1038111610,2019-08-07 16:03:09,20173847,0.0,NaN
3,2019-08-08 19:08:57,600.0,2017111153,3600.0,男表,,雷达,1038111556,2017111152,0,12072,1,0.0,,0,1098119583,2019-08-08 19:08:57,20173649,0.0,NaN
4,2019-08-21 16:26:36,333.0,2017111146,3333.0,手包,,乔丹,1098119585,2017111137,0,12823,1,0.0,,0,1098119583,2019-08-21 16:26:36,20174763,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,2019-12-02 18:52:49,10000.0,2017111142,15000.0,公文包,,葆蝶家BV男士公文包手提包编织包,1098119554,2017111137,0,20175104_1098119554,1,8000.0,,0,1098119650,2019-12-02 18:52:49,20175104,NaN,0.0
717,2019-12-02 18:53:03,36000.0,2017111138,66000.0,单肩包,,古驰Gucci男士单肩包斜挎包,1098119554,2017111137,0,20175651_1098119554,1,31000.0,,0,1098119650,2019-12-02 18:53:03,20175651,NaN,0.0
718,2019-11-28 21:05:48,6345.0,2017111165,11345.0,围巾,,路易威登,1098119507,2017111162,0,20175753_1098119507,2,4345.0,,0,1098119649,2019-12-02 20:51:15,20175753,NaN,0.0
719,2019-12-02 20:51:20,6399.0,2017111165,11399.0,围巾,,路易威登,1098119507,2017111162,0,20175746_1098119507,1,4399.0,,0,1098119678,2019-12-02 20:51:20,20175746,NaN,0.0


In [7]:
pv_df.sort_values('createtime', ascending=0)

,userid,bbsid,batch,updatetime,createtime,bbs_createtime,userid_bbs,sug_type,isBusiAuth,useridentity,sug_version,sug_total
42,1098117798,20175785,9,2019-12-02 20:43:10,2019-12-02 20:43:10,2019-11-29 21:27:02,1038111556,3,1,2,0,0.0000
83,1098117798,20175783,2,2019-12-02 20:43:10,2019-12-02 20:43:10,2019-11-29 21:06:56,1038111556,3,1,2,0,0.0000
41,1098117798,20175789,9,2019-12-02 20:43:10,2019-12-02 20:43:10,2019-11-29 21:50:38,1098119570,3,1,2,0,0.0000
37,1098119554,20175794,10,2019-12-02 19:45:27,2019-12-02 19:45:27,2019-12-02 15:10:06,1098119569,3,1,2,0,0.0000
82,1098119554,20175793,3,2019-12-02 19:45:27,2019-12-02 19:45:27,2019-12-02 15:08:46,1098119569,3,1,2,0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
14,1098119703,20175750,1,2019-11-29 20:59:01,2019-11-29 20:59:01,2019-11-28 19:57:24,1098119688,2,0,1,1,21.5755
8,1098119703,20175749,1,2019-11-29 20:59:01,2019-11-29 20:59:01,2019-11-28 19:56:23,1098119688,2,0,1,1,18.6185
7,1098119703,20175737,5,2019-11-29 20:59:01,2019-11-29 20:59:01,2019-11-28 19:20:05,1038111611,2,0,1,1,18.3145
57,1098119703,20175746,5,2019-11-29 20:59:01,2019-11-29 20:59:01,2019-11-28 19:52:04,1098119678,2,0,1,1,19.1305


In [8]:
es.search?